In [ ]:
# Data Processing from base data to dataset for Truth-in-Tax vs Statutory rate counterfactual

In [6]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")

# drop needless columns
df_clean = df_clean[['Tax Year', 'County Name', 'Entity Name', 'Entity Code', 'County Code', 'Entity Type', 'Tax Rate' , 
                 'Total Taxable Property', 'Revenue, Total']]

In [ ]:
# Compute rates without the Truth in Tax constraint

df_clean["Counterfactual Tax Rate"] = 0

years = df_clean["Tax Year"].unique().tolist()
entities = df_clean['Entity Name'].unique().tolist()

for year in years:
    for entities in entities:


In [ ]:
# Export to JSON
# df_clean.to_json('../../public/data/counterfactual_data.json')